Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig, AdamW, Trainer, TrainingArguments
from datasets import Dataset, load_metric

# Load your dataset
data = pd.read_csv('useeio_dataset.csv')

# Preprocess the data (example, adjust according to your dataset)
# Assume 'text' is the column with textual descriptions and 'emission' is the target variable for regression tasks.
features = data['text_description'].tolist()
emissions = data['emission_quantity'].values

# Split the data into train and test sets
train_texts, test_texts, train_emissions, test_emissions = train_test_split(features, emissions, test_size=0.2, random_state=42)

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Convert to torch dataset
class EmissionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, emissions):
        self.encodings = encodings
        self.emissions = emissions

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.emissions[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.emissions)

train_dataset = EmissionDataset(train_encodings, train_emissions)
test_dataset = EmissionDataset(test_encodings, test_emissions)


Model Definition - 1.Traditional Supervised Fine-Tuning (TSFT)

In [ ]:
# Fine-tuning BERT for regression
class BertRegression(nn.Module):
    def __init__(self):
        super(BertRegression, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.regressor(pooled_output)

# Initialize the model
model = BertRegression()

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {
        'mse': mean_squared_error(p.label_ids, p.predictions),
        'rmse': mean_squared_error(p.label_ids, p.predictions, squared=False),
        'mae': mean_absolute_error(p.label_ids, p.predictions),
        'r2': r2_score(p.label_ids, p.predictions)
    }
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("TSFT Results:", results)


Transfer Learning with Pre-trained Models (TLPM)

In [ ]:
# For TLPM, you can use a pre-trained BERT model and fine-tune it directly as shown above.
# The same code for TSFT applies to TLPM if you load a pre-trained model.

# Initialize the model with pre-trained weights
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {
        'mse': mean_squared_error(p.label_ids, p.predictions),
        'rmse': mean_squared_error(p.label_ids, p.predictions, squared=False),
        'mae': mean_absolute_error(p.label_ids, p.predictions),
        'r2': r2_score(p.label_ids, p.predictions)
    }
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("TLPM Results:", results)


Multi-Task Learning (MTL)

In [ ]:
# For MTL, create a multi-task BERT model. Assume the additional tasks are classification tasks.

class MultiTaskBERT(nn.Module):
    def __init__(self):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)  # adjust num_classes

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        regression_output = self.regressor(pooled_output)
        classification_output = self.classifier(pooled_output)
        return regression_output, classification_output

# Initialize the model
model = MultiTaskBERT()

# Define a custom training loop for MTL
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion_regression = nn.MSELoss()
criterion_classification = nn.CrossEntropyLoss()

for epoch in range(training_args.num_train_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_regression = batch['labels_regression'].to(device)
        labels_classification = batch['labels_classification'].to(device)
        
        outputs = model(input_ids, attention_mask)
        loss_regression = criterion_regression(outputs[0], labels_regression)
        loss_classification = criterion_classification(outputs[1], labels_classification)
        loss = loss_regression + loss_classification
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
# ... (custom evaluation code similar to the above)


Domain-Adversarial Training (DAT)

In [ ]:
# For DAT, create a domain classifier alongside the main BERT model.

class DomainAdversarialBERT(nn.Module):
    def __init__(self):
        super(DomainAdversarialBERT, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.domain_classifier = nn.Linear(self.bert.config.hidden_size, num_domains)  # adjust num_domains

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        regression_output = self.regressor(pooled_output)
        domain_output = self.domain_classifier(pooled_output)
        return regression_output, domain_output

# Initialize the model
model = DomainAdversarialBERT()

# Define a custom training loop for DAT
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion_regression = nn.MSELoss()
criterion_domain = nn.CrossEntropyLoss()

for epoch in range(training_args.num_train_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_regression = batch['labels_regression'].to(device)
        labels_domain = batch['labels_domain'].to(device)
        
        outputs = model(input_ids, attention_mask)
        loss_regression = criterion_regression(outputs[0], labels_regression)
        loss_domain = criterion_domain(outputs[1], labels_domain)
        loss = loss_regression + loss_domain
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
# ... (custom evaluation code similar to the above)


Evaluation and plotting results

In [ ]:
def evaluate_model(model, test_dataset):
    model.eval()
    predictions = []
    true_labels = []
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask)
            predictions.append(outputs.cpu().numpy())
            true_labels.append(labels.cpu().numpy())
    
    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)

    mse = mean_squared_error(true_labels, predictions)
    rmse = mean_squared_error(true_labels, predictions, squared=False)
    mae = mean_absolute_error(true_labels, predictions)
    r2 = r2_score(true_labels, predictions)

    return mse, rmse, mae, r2

# Evaluate each model
tsft_results = evaluate_model(tsft_model, test_dataset)
tlpm_results = evaluate_model(tlpm_model, test_dataset)
mtl_results = evaluate_model(mtl_model, test_dataset)
dat_results = evaluate_model(dat_model, test_dataset)

# Print the results
print("TSFT Results:", tsft_results)
print("TLPM Results:", tlpm_results)
print("MTL Results:", mtl_results)
print("DAT Results:", dat_results)


In [ ]:
import matplotlib.pyplot as plt

methods = ['TSFT', 'TLPM', 'MTL', 'DAT']
mse_values = [tsft_results[0], tlpm_results[0], mtl_results[0], dat_results[0]]
rmse_values = [tsft_results[1], tlpm_results[1], mtl_results[1], dat_results[1]]
mae_values = [tsft_results[2], tlpm_results[2], mtl_results[2], dat_results[2]]
r2_values = [tsft_results[3], tlpm_results[3], mtl_results[3], dat_results[3]]

fig, ax = plt.subplots(2, 2, figsize=(12, 10))

ax[0, 0].bar(methods, mse_values)
ax[0, 0].set_title('Mean Squared Error (MSE)')

ax[0, 1].bar(methods, rmse_values)
ax[0, 1].set_title('Root Mean Squared Error (RMSE)')

ax[1, 0].bar(methods, mae_values)
ax[1, 0].set_title('Mean Absolute Error (MAE)')

ax[1, 1].bar(methods, r2_values)
ax[1, 1].set_title('R-squared (R2)')

plt.tight_layout()
plt.show()
